# Use the VLM endpoint

In [ ]:
import base64
import cv2
import numpy as np
import os
import pyheif

from typing import Optional

from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from openai import AzureOpenAI, ChatCompletion


# Function to encode an image
def encode_image(image_path: str):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# Function to encode and resize an image
def encode_and_resize(image_path: str, max_size: int = 640):
    if image_path.endswith(".HEIC"):
        img = read_heic_to_numpy(image_path)
    else:
        img = cv2.imread(image_path)
    scale_factor = max_size / max(img.shape)
    img = cv2.resize(img, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LINEAR)
    _, im_arr = cv2.imencode('.jpg', img)  # im_arr: image in Numpy one-dim array format.
    im_bytes = im_arr.tobytes()
    return base64.b64encode(im_bytes).decode("utf-8")

# Function to read .HEIC image format
def read_heic_to_numpy(file_path: str):
    heif_file = pyheif.read(file_path)
    data = heif_file.data
    if heif_file.mode == "RGB":
        numpy_array = np.frombuffer(data, dtype=np.uint8).reshape(
            heif_file.size[1], heif_file.size[0], 3)
    elif heif_file.mode == "RGBA":
        numpy_array = np.frombuffer(data, dtype=np.uint8).reshape(
            heif_file.size[1], heif_file.size[0], 4)
    else:
        raise ValueError("Unsupported HEIC color mode")
    return numpy_array


endpoint = "https://oai-aip-cv-ont-sdc.openai.azure.com/"
model_name = "gpt-4o"
deployment = "gpt-4o"
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    azure_ad_token_provider=token_provider,
)

## Solution Team 2

In [ ]:
SYSTEM_PROMPT = """
Je bent een AI-assistent die foto's van de openbare ruimte beoordeelt op basis van de CROW-systematiek gericht op voetgangersveiligheid, met een focus op struikelgevaar. Analyseer de foto op de volgende aspecten:

1. **Oneffenheden in het oppervlak**: Identificeer hoogteverschillen, losse of middende elementen in de verharding, of gaten.
2. **Continuïteit van het loopvlak**: Controleer of het loopvlak onderbroken wordt en of dit voetgangers dwingt om obstakels te vermijden, uit balans te raken, of ongebruikelijke stappen te zetten.
3. **Materiaal en conditie**: Beoordeel materiaalbeschadiging zoals scheuren, verzakkingen, of gladde oppervlakken die struikelrisico’s kunnen verhogen.
4. **Omgevingsfactoren**: Houd rekening met omgevingskenmerken zoals slechte verlichting, scherpe bochten of verwaarloosde plekken die risico's kunnen verergeren.  

Geef een prioriteitsscore:  
- **Laag** risico: Klein of verwaarloosbaar struikelgevaar, interventie kan wachten. Dit is bijvoorbeeld het geval wanneer de oneffenheid kleiner is dan 1cm.
- **Middel** risico: Aandacht vereist, plan remedie op middellange termijn. Dit is bijvoorbeeld het geval wanneer de oneffenheid tussen de 1cm en 3cm is.
- **Hoog** risico: Directe actie noodzakelijk vanwege significante kans op letsel. Dit is bijvoorbeeld het geval wanneer de oneffenheid groter is dan 3cm.

Formatteer je analyse als een inspectierapport.

Inspectierapport
---
**Omschrijving van het probleem:** [Beknopte omschrijving van het belangrijkste probleem, bijvoorbeeld "Losliggende tegel", "Boomwortelschade", "Gat"].
**Omgevingsfactoren:** [Bijv. Slecht zicht, aanwezige obstakels, weersinvloed]
**Hoogteverschil:** [Schatting van het hoogteverschil in cm]
**Beoordeling van risico:** {Laag, Middel, Hoog}
**Toelichting:** [Geef een beknopte technische verklaring voor de risico score, gebruikmakend van je analyse. Refereer specifieke elementen in de foto. Voorbeeld: "De foto toont een losliggende stoeptegel in het midden van de looproute. De oneffenheid is groter dan 3cm en valt dus onder hoog risico."] 
---

Als er geen duidelijk risico zichtbaar is, geef dan "Geen significant risico" als **Omschrijving van het probleem** en geef als prioriteit "Laag".
"""

USER_PROMPT = """
Genereer een inspectierapport voor deze foto.
"""

def get_response(base64_image: str):
    """Each team can customize this function as they want to get the best results for their use case.

    Parameters
    ----------
    base64_image : str
        Base64 encoded image

    Returns
    -------
    str
        Text response from the model according to the prompt
    """
    response = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": SYSTEM_PROMPT,
            },
            {
                "role": "user",
                "content": [
                    { "type": "text", "text": USER_PROMPT },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}",
                            "detail": "high", # reduces token usage
                        },
                    },
                ],
            }
        ],
        max_tokens=4096,
        temperature=0.2,
        model=deployment
    )
    return response.choices[0].message.content